In [37]:
import tequila as tq
import numpy as np
from tequila.hamiltonian import QubitHamiltonian, paulis
from tequila.grouping.binary_rep import BinaryHamiltonian

The following examples shows how to partition a given Hamiltonian into commuting parts and how to find the unitary transformation needed to transform the commuting terms into qubit-wise commuting form that is easy to measure. 

The Hamiltonian is simply 
$$ H = \sigma_z(0)\sigma_z(1) + \sigma_y(0)\sigma_y(1) + \sigma_x(0)\sigma_x(1) + \sigma_x(0)$$
where $\sigma_z(0)\sigma_z(1)$, $\sigma_y(0)\sigma_y(1)$ does not commute with $\sigma_x(0)$, so two separate measurements are needed.

In [38]:
H = paulis.Z(0) * paulis.Z(1) + paulis.Y(0) * paulis.Y(1) + \
    paulis.X(0) * paulis.X(1) + paulis.X(0) 

Here we use the binary representation of the Hamiltonian for partitioning. The method commuting_groups gets back a list of BinaryHamiltonian whose terms are mutually commuting. 

Call to_qubit_hamiltonian to visualize.

In [50]:
binary_H = BinaryHamiltonian.init_from_qubit_hamiltonian(H)
commuting_parts = binary_H.commuting_groups()

In [40]:
print(len(commuting_parts)) # Number of measurements needed
print(commuting_parts[0].to_qubit_hamiltonian())
print(commuting_parts[1].to_qubit_hamiltonian())

2
+0.0000i+1.0000X(0)X(1)+1.0000X(0)
+0.0000i+1.0000Z(0)Z(1)+1.0000Y(0)Y(1)


The second group of terms $H_2$ are not currently qubit-wise commuting and cannot be directly measured on current hardware. They require further unitary transformation $U$ to become qubit-wise commuting. The following code identifies two bases (list of BinaryPauliString) that encodes the unitary transformation as
$$ U = \prod_i \frac{1}{2} (\text{old_basis}[i] + \text{new_basis}[i])$$
such that $UH_2U$ is qubit-wise commuting.

In [49]:
qubit_wise_parts, old_basis, new_basis = commuting_parts[1].get_qubit_wise()

In [46]:
def display_basis(basis):
    for term in basis:
        print(QubitHamiltonian.init_from_paulistring(term.to_pauli_strings()))
print('Old Basis')
display_basis(old_basis)
print('\nNew Basis')
display_basis(new_basis)

Old Basis
+1.0000Y(0)Y(1)
+1.0000X(0)X(1)

New Basis
+1.0000X(0)
+1.0000Y(1)


The transfromed term $UH_2U$ is qubit-wise commuting. 

In [47]:
print(qubit_wise_parts.to_qubit_hamiltonian())

+0.0000i-1.0000X(0)Y(1)+1.0000X(0)
